# Lightgbm model test notebook

In [4]:
# !brew install cmake libomp
# !pip install lightgbm

Running `brew update --auto-update`...
==> Homebrew collects anonymous analytics.
Read the analytics documentation (and how to opt-out) here:
  https://docs.brew.sh/Analytics
No analytics have been recorded yet (nor will be during this `brew` run).

==> Homebrew is run entirely by unpaid volunteers. Please consider donating:
  https://github.com/Homebrew/brew#donations

==> Auto-updated Homebrew!
Updated 2 taps (homebrew/core and homebrew/cask).
==> New Formulae
actions-batch              jtbl                       prjtrellis
appwrite                   k8sgpt                     proto
asmfmt                     kin                        protoc-gen-js
autobrr                    kiota                      ratchet
bluez                      ktfmt                      rathole
bpftop                     kubeshark                  rattler-build
c-blosc2                   kubetui                    rawdog
c3c                        libnsbmp                   redict
c4core                   

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # statistical data visualization

import lightgbm as lgb


In [2]:
X_train_full_df = pd.read_csv("X_train_NMF_topics.csv")
X_test_full_df = pd.read_csv("X_test_NMF_topics.csv")

In [3]:
pd.set_option('display.max_columns', 500)

## In this test the target variable will be "weighted rating"

### Definining X_train, X_test, y_train, y_test

In [4]:
columns_to_drop = ["Title","description","authors","image",
                   "previewLink","publisher","infoLink","categories",
                   "index","reviews number","average rating",
                   "median rating","min review date_x","min review date_y",
                   "weighted rating","date","description_language","tokens"]

X_train = X_train_full_df.drop(columns_to_drop, axis = 1)
X_test = X_test_full_df.drop(columns_to_drop, axis = 1)

# change the year feature to an integer
X_train["year"] = X_train["year"].astype(int)
X_test["year"] = X_test["year"].astype(int)

y_train = X_train_full_df["weighted rating"]
y_test = X_test_full_df["weighted rating"]

### Hyperparameters for lightgbm

In [6]:
hyper_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': ['l1','l2'],
    'learning_rate': 0.005,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.7,
    'bagging_freq': 10,
    'verbose': 0,
    "max_depth": 8,
    "num_leaves": 128,  
    "max_bin": 512,
    "num_iterations": 100000
}

In [7]:
# hyper_params = {
#     'task': 'train',
#     'boosting_type': 'gbdt',
#     'objective': 'regression',
#     'metric': ['l1','l2'],
#     'learning_rate': 0.1, # Changed to default
#     'verbose': 0,
#     "max_depth": 8,
#     "num_leaves": 10,  
# }

hyper_params = {
    'task': 'train',
    'objective': 'regression',
    'metric': ['l1','l2'],
    'learning_rate': 0.1, # Changed to default
    'verbose': 0,
    "max_depth": 8,
    "num_leaves": 10,  
}

In [ ]:
# # Constructing the lgbdataset object
# lgb_train = lgb.Dataset(X_train, y_train)
# lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)


# Instantiating and fitting the lightgbm model
# gbm = lgb.LGBMRegressor(hyper_params)

gbm = lgb.LGBMRegressor(
    boosting_type= 'gbdt',
    n_estimators=100,
    learning_rate= 0.1, # Changed to default
    num_boost_round=100,
    max_depth=5
)

# gbm.fit(X_train, y_train,
#         eval_set=[(X_test, y_test)],
#         eval_metric='l1',
#         callbacks=[
#         lgb.early_stopping(stopping_rounds=3),
#     ])

gbm.fit(X_train, y_train)

/Users/meganguidry/miniconda3/lib/python3.11/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] num_iterations is set=100, num_boost_round=100 will be ignored. Current value: num_iterations=100
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


In [6]:
# # Constructing the lgbdataset object
# lgb_train = lgb.Dataset(X_train, y_train)
# lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

# # changed early stopping rounds from 30 to 5

# lgb_model = lgb.train(hyper_params,
#                       train_set=lgb_train,
#                       valid_sets=lgb_eval,
#                       callbacks=[lgb.early_stopping(stopping_rounds=5)]
#                      )

In [ ]:
y_pred = gbm.predict(X_train, num_iteration=bgm.best_iteration_)

In [ ]:
# Basic RMSE
print('The rmse of prediction is:', round(mean_squared_log_error(y_pred, y_train) ** 0.5, 5))

# CATBOOST!

In [78]:
# Define X_train, X_test, y_train, y_test for the datasets with AND without the book cover encodings:

# Import NMF_topics datafiles (these do not include image embeddings
X_train_full_df = pd.read_csv("X_train_NMF_topics.csv")
X_test_full_df = pd.read_csv("X_test_NMF_topics.csv")

In [79]:
# Define the train/test split for the datasets that do not 
# include image embeddings:

# Identify the columns that we do not want in our feature set:
columns_to_drop = ["Title","description","authors","image",
                   "previewLink","publisher","infoLink","categories",
                   "reviews number","average rating",
                   "median rating","min review date_x","min review date_y",
                   "weighted rating","date","description_language","tokens"]

# Remove the unwanted features:
X_train_NOIMAGE = X_train_full_df.drop(columns_to_drop, axis = 1)
X_test_NOIMAGE = X_test_full_df.drop(columns_to_drop, axis = 1)

# change the year feature to an integer
X_train_NOIMAGE["year"] = X_train["year"].astype(int)
X_test_NOIMAGE["year"] = X_test["year"].astype(int)

# Set the target variable
y_train = X_train_full_df["weighted rating"]
y_test = X_test_full_df["weighted rating"]

# remember to drop index!!!

In [80]:
X_train_NOIMAGE.head(2)

,index,year,nostalgia,self-published/debut,story/anthology,womens_fiction,childrens_books,classic,family_drama,digital_books/recreations,reproduced,murder_mystery,reprint,bestselling_author,romance,unkonwn,teen,novel,world/war/historical_fiction,unknown,young_adult,coming_of_age
0,3,2005,0.007321,0.000000,0.000000,0.0,0.000000,0.0,0.017810,0.0,0.009671,0.015191,0.001315,0.0,0.040636,0.0,0.000214,0.000000,0.010339,0.0,0.0,0.000000
1,24,2001,0.017931,0.000181,0.016551,0.0,0.011345,0.0,0.004711,0.0,0.000000,0.005422,0.000000,0.0,0.009457,0.0,0.000000,0.006398,0.000197,0.0,0.0,0.000621


In [81]:
#Import dataset with embeddings
title_embeddings_df = pd.read_pickle(
    'original_data/English_fiction_pre_PCA_3_with_av_pool_embeddings')

In [82]:
# Select only necessary features (features needed for merging)
title_embeddings_df = title_embeddings_df[["index", "Title", "Embedding"]]
title_embeddings_df = title_embeddings_df.set_index("index")

In [83]:
# Convert array to columns
embeddings_df = title_embeddings_df['Embedding'].apply(pd.Series)
embeddings_df = embeddings_df.add_prefix('image_')

In [84]:
# Now include images in the dataset:
X_train_withIMAGE = X_train_NOIMAGE.merge(
    embeddings_df,
    left_on='index', 
    right_on=embeddings_df.index,
    how = 'inner')

X_test_withIMAGE = X_test_NOIMAGE.merge(
    embeddings_df,
    left_on='index', 
    right_on=embeddings_df.index,
    how = 'inner')

y_train_NOIMAGE = X_train_full_df["weighted rating"]
y_test_NOIMAGE = X_test_full_df["weighted rating"]

In [85]:
# Drop index from data before inserting into ML models:
X_train_NOIMAGE = X_train_NOIMAGE.drop("index", axis = 1)
X_test_NOIMAGE = X_test_NOIMAGE.drop("index", axis = 1)

X_train_withIMAGE = X_train_withIMAGE.drop("index", axis = 1)
X_test_withIMAGE = X_test_withIMAGE.drop("index", axis = 1)

In [ ]:
# !pip install catboost

In [91]:
import catboost as cb
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Build and fit a CatBoost regressor
reg_cat = cb.CatBoostRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

reg_cat.fit(X_train_withIMAGE, y_train)

# Calculate the predictions on the test set
pred =reg_cat.predict(X_test_withIMAGE)

# Evaluate the performance using the RMSE
rmse_cat = np.sqrt(mean_squared_error(y_test, pred))
mse_cat = mean_squared_error(y_test, pred)
print('MSE (CatBoost): {:.5f}'.format(mse_cat))
print('RMSE (CatBoost): {:.5f}'.format(rmse_cat))

0:	learn: 0.2070900	total: 3.75ms	remaining: 371ms
1:	learn: 0.2066570	total: 7.15ms	remaining: 350ms
2:	learn: 0.2062758	total: 11.4ms	remaining: 369ms
3:	learn: 0.2059653	total: 15ms	remaining: 360ms
4:	learn: 0.2056813	total: 18.7ms	remaining: 355ms
5:	learn: 0.2054078	total: 22.4ms	remaining: 352ms
6:	learn: 0.2051556	total: 26.5ms	remaining: 352ms
7:	learn: 0.2049790	total: 30ms	remaining: 345ms
8:	learn: 0.2047804	total: 33.3ms	remaining: 337ms
9:	learn: 0.2046048	total: 36.7ms	remaining: 330ms
10:	learn: 0.2044786	total: 40.1ms	remaining: 325ms
11:	learn: 0.2043179	total: 43.6ms	remaining: 319ms
12:	learn: 0.2042060	total: 46.9ms	remaining: 314ms
13:	learn: 0.2040321	total: 50.2ms	remaining: 308ms
14:	learn: 0.2039142	total: 53.4ms	remaining: 303ms
15:	learn: 0.2037998	total: 56.5ms	remaining: 297ms
16:	learn: 0.2036697	total: 59.9ms	remaining: 293ms
17:	learn: 0.2035568	total: 63.3ms	remaining: 288ms
18:	learn: 0.2034547	total: 66.6ms	remaining: 284ms
19:	learn: 0.2033778	total

In [96]:
import catboost as cb
from sklearn.metrics import mean_squared_error, mean_absolute_error

def catboost_execute(X_train,
                     X_test,
                     y_train,
                     y_test,
                     num_estimators,
                     learn_rate,
                     depth_max, 
                     ):
    solution = {}
    # Build and fit a CatBoost regressor
    reg_cat = cb.CatBoostRegressor(n_estimators=num_estimators, 
                                   learning_rate=learn_rate, 
                                   max_depth=depth_max, 
                                   random_state=42)
    
    reg_cat.fit(X_train, y_train)
    
    # Calculate the predictions on the test set
    pred =reg_cat.predict(X_test)
    
    # Evaluate the performance using the RMSE
    rmse_cat = np.sqrt(mean_squared_error(y_test, pred))
    mse_cat = mean_squared_error(y_test, pred)
    mae_cat = mean_absolute_error(y_test, pred)
    print('MSE (CatBoost): {:.5f}'.format(mse_cat))
    print('RMSE (CatBoost): {:.5f}'.format(rmse_cat))
    
    solution["MSE"] = mse_cat
    solution["RMSE"] = rmse_cat
    solution["MAE"] = mae_cat
    
    return solution

In [99]:
catboost_execute(X_train_withIMAGE,
                 X_test_withIMAGE,
                 y_train,
                 y_test,
                 100,
                 0.1,
                 3)

0:	learn: 0.2070900	total: 5.69ms	remaining: 564ms
1:	learn: 0.2066570	total: 10.6ms	remaining: 522ms
2:	learn: 0.2062758	total: 14.5ms	remaining: 469ms
3:	learn: 0.2059653	total: 18ms	remaining: 432ms
4:	learn: 0.2056813	total: 21.5ms	remaining: 408ms
5:	learn: 0.2054078	total: 25ms	remaining: 392ms
6:	learn: 0.2051556	total: 28.6ms	remaining: 380ms
7:	learn: 0.2049790	total: 31.8ms	remaining: 365ms
8:	learn: 0.2047804	total: 35.2ms	remaining: 356ms
9:	learn: 0.2046048	total: 38.6ms	remaining: 347ms
10:	learn: 0.2044786	total: 42ms	remaining: 340ms
11:	learn: 0.2043179	total: 45.5ms	remaining: 334ms
12:	learn: 0.2042060	total: 49.1ms	remaining: 329ms
13:	learn: 0.2040321	total: 52.5ms	remaining: 323ms
14:	learn: 0.2039142	total: 55.8ms	remaining: 316ms
15:	learn: 0.2037998	total: 59ms	remaining: 310ms
16:	learn: 0.2036697	total: 62.4ms	remaining: 304ms
17:	learn: 0.2035568	total: 65.7ms	remaining: 299ms
18:	learn: 0.2034547	total: 69.1ms	remaining: 295ms
19:	learn: 0.2033778	total: 72

{'MSE': 0.04286034575956498,
 'RMSE': 0.20702740340246018,
 'MAE': 0.1331532463785843}

In [ ]:
# DELETE BELOW

In [37]:
#Import dataset with embeddings
title_embeddings_df = pd.read_pickle(
    'original_data/English_fiction_pre_PCA_3_with_av_pool_embeddings')

In [39]:
title_embeddings_df = title_embeddings_df[["index", "Title", "Embedding"]]
title_embeddings_df = title_embeddings_df.set_index("index")

In [41]:
title_embeddings_df.head()

,Title,Embedding
index,,
3,Whispers of the Wicked Saints,"[0.5179044, -0.7533603, -1.1291503, -0.4418345..."
24,The Forbidden Stories of Marta Veneranda,"[0.706188, -0.4773652, -0.17887038, 0.07989502..."
42,Tess and the Highlander,"[2.294651, -0.24902871, -0.6188333, -0.7722471..."
49,"Eight Men And A Lady (Elizabeth Sinclair, Harl...","[0.37794992, -0.6178984, -0.81393754, -0.66795..."
73,Night World: Daughters Of Darkness,"[0.34032565, -2.1706967, -0.21470371, -0.10447..."


In [45]:
# Convert array to columns
embeddings_df = title_embeddings_df['Embedding'].apply(pd.Series)
embeddings_df = embeddings_df.add_prefix('image_')

In [46]:
embeddings_df.head()

,image_0,image_1,image_2,image_3,image_4,image_5,image_6,image_7,image_8,image_9,image_10,image_11,image_12,image_13,image_14,image_15,image_16,image_17,image_18,image_19,image_20,image_21,image_22,image_23,image_24,image_25,image_26,image_27,image_28,image_29,image_30,image_31,image_32,image_33,image_34,image_35,image_36,image_37,image_38,image_39,image_40,image_41,image_42,image_43,image_44,image_45,image_46,image_47,image_48,image_49,image_50,image_51,image_52,image_53,image_54,image_55,image_56,image_57,image_58,image_59,image_60,image_61,image_62,image_63,image_64,image_65,image_66,image_67,image_68,image_69,image_70,image_71,image_72,image_73,image_74,image_75,image_76,image_77,image_78,image_79,image_80,image_81,image_82,image_83,image_84,image_85,image_86,image_87,image_88,image_89,image_90,image_91,image_92,image_93,image_94,image_95,image_96,image_97,image_98,image_99,image_100,image_101,image_102,image_103,image_104,image_105,image_106,image_107,image_108,image_109,image_110,image_111,image_112,image_113,image_114,image_115,image_116,image_117,image_118,image_119,image_120,image_121,image_122,image_123,image_124,image_125,image_126,image_127,image_128,image_129,image_130,image_131,image_132,image_133,image_134,image_135,image_136,image_137,image_138,image_139,image_140,image_141,image_142,image_143,image_144,image_145,image_146,image_147,image_148,image_149,image_150,image_151,image_152,image_153,image_154,image_155,image_156,image_157,image_158,image_159,image_160,image_161,image_162,image_163,image_164,image_165,image_166,image_167,image_168,image_169,image_170,image_171,image_172,image_173,image_174,image_175,image_176,image_177,image_178,image_179,image_180,image_181,image_182,image_183,image_184,image_185,image_186,image_187,image_188,image_189,image_190,image_191,image_192,image_193,image_194,image_195,image_196,image_197,image_198,image_199,image_200,image_201,image_202,image_203,image_204,image_205,image_206,image_207,image_208,image_209,image_210,image_211,image_212,image_213,image_214,image_215,image_216,image_217,image_218,image_219,image_220,image_221,image_222,image_223,image_224,image_225,image_226,image_227,image_228,image_229,image_230,image_231,image_232,image_233,image_234,image_235,image_236,image_237,image_238,image_239,image_240,image_241,image_242,image_243,image_244,image_245,image_246,image_247,image_248,image_249,image_250,image_251,image_252,image_253,image_254,image_255
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3,0.517904,-0.753360,-1.129150,-0.441835,0.754806,-1.094556,-0.261775,-0.007399,-0.871388,-1.445458,-0.081653,-1.412558,-1.305154,0.233091,-0.665063,-1.042683,2.391319,0.911486,-0.794151,-0.602357,2.293958,1.406600,0.398244,-0.217079,1.811218,1.177913,0.615737,0.001465,0.826524,-1.053041,1.865097,-0.329853,-1.986748,0.485420,1.852550,0.390180,0.773877,-2.334943,-0.789028,0.908791,-0.434157,0.340118,0.071087,0.625080,1.140045,-0.094658,-0.214363,0.439892,-0.799621,1.355317,1.563076,-1.155384,-0.184857,0.988588,1.238690,0.031409,0.274971,-1.422949,1.297558,-1.561182,1.928400,0.882364,0.281398,-0.550819,-0.543688,-1.126661,-1.527737,-0.921328,-0.530724,0.600521,-0.061235,0.678182,2.450924,-0.712121,0.159680,1.440127,0.096627,-1.509313,1.153900,-0.151364,-0.186872,0.570882,-1.201824,-2.120494,1.360526,0.200105,1.012025,-1.082366,1.815913,0.038653,-0.111678,-0.148673,-0.718856,-2.552412,2.024915,-0.404083,-0.174378,-0.995185,0.328322,0.752824,-1.182382,0.846173,1.174780,-0.595725,0.725202,-0.355794,1.197939,-0.738304,-0.374900,0.933315,-0.493223,1.059865,-1.043250,-1.014508,0.726310,-0.970106,0.485822,2.318572,-1.372488,-0.748245,0.597242,-0.501010,-0.205499,-1.586742,-0.445546,-2.101681,0.482173,0.053082,0.171499,-1.871133,-0.277225,0.851118,2.443469,1.086347,2.681574,-2